# Day 9: Sensor Boost
This adds a couple more features to the Day 7 Intcode computer, mostly around the Relative Base Offset concept. We also refactor and simplify a lot of the class structure from Day 7.

Import `defaultdict` to make the input memory array more growable.

In [1]:
from collections import defaultdict

Here we simplify the `Instruction` class to remove any logic besides the basic reading from params and writing to memory. The logic for handling `modes` and for what kind of action to take upon completion are both left up to the caller.

In [2]:
class Instruction:
    def __init__(self, opcode, length, param_locations, param_values, memory):
        self.opcode = opcode
        self.length = length
        self.param_locations = param_locations
        self.param_values = param_values
        self.memory = memory
        
    def get_val(self, i):
        return self.param_values[i]
    
    def set_val(self, i, val):
        self.memory[self.param_locations[i]] = val

We also simplify the individual `Instruction` subclasses by not requiring them to return anything.

In [3]:
class Add(Instruction):
    def call(self):
        self.set_val(2, self.get_val(0) + self.get_val(1))

In [4]:
class Multiply(Instruction):
    def call(self):
        self.set_val(2, self.get_val(0) * self.get_val(1))

In [5]:
class Input(Instruction):
    def call(self, input_value):
        self.set_val(0, input_value)

In [6]:
class Output(Instruction):
    def call(self):
        return self.get_val(0)

In [7]:
class JumpIfTrue(Instruction):
    def call(self):
        return self.get_val(1) if self.get_val(0) else None

In [8]:
class JumpIfFalse(Instruction):
    def call(self):
        return self.get_val(1) if not self.get_val(0) else None

In [9]:
class LessThan(Instruction):
    def call(self):
        val = 1 if self.get_val(0) < self.get_val(1) else 0
        self.set_val(2, val)

In [10]:
class Equals(Instruction):
    def call(self):
        val = 1 if self.get_val(0) == self.get_val(1) else 0
        self.set_val(2, val)

This new Instruction returns a value to the caller so it can adjust the `relative_base`.

In [11]:
class RelativeBaseOffset(Instruction):
    def call(self):
        return self.get_val(0)

In [12]:
class Halt(Instruction):
    pass

Here we do an overhaul to the `IntcodeComputer` class (f.k.a. `Program`). All the global constants are now in here, as is the `create_instruction()` method, which also handles `mode` interpretation.

One of the new requirements is that the `memory` we keep track of needs to be able to be arbitrarily expanded as the program executes. To do this we convert the data structure of the `memory` from a list to a `defaultdict`.

In [13]:
class IntcodeComputer:
    # Instruction opcodes
    ADD = 1
    MULTIPLY = 2
    INPUT = 3
    OUTPUT = 4
    JUMP_IF_TRUE = 5
    JUMP_IF_FALSE = 6
    LESS_THAN = 7
    EQUALS = 8
    RELATIVE_BASE_OFFSET = 9
    HALT = 99
    
    LENGTHS = {
        ADD: 4,
        MULTIPLY: 4,
        INPUT: 2,
        OUTPUT: 2,
        JUMP_IF_TRUE: 3,
        JUMP_IF_FALSE: 3,
        LESS_THAN: 4,
        EQUALS: 4,
        RELATIVE_BASE_OFFSET: 2,
        HALT: 1,
    }
    
    INSTRUCTIONS = {
        ADD: Add,
        MULTIPLY: Multiply,
        INPUT: Input,
        OUTPUT: Output,
        JUMP_IF_TRUE: JumpIfTrue,
        JUMP_IF_FALSE: JumpIfFalse,
        LESS_THAN: LessThan,
        EQUALS: Equals,
        RELATIVE_BASE_OFFSET: RelativeBaseOffset,
        HALT: Halt,
    }
    
    POSITION_MODE = '0'
    IMMEDIATE_MODE = '1'
    RELATIVE_MODE = '2'
    
    def __init__(self, program):
        self.memory = program.copy()
        self.memory_pointer = 0
        self.relative_base = 0
        
    def create_instruction(self):
        op = str(self.memory[self.memory_pointer])
        opcode = int(op[-2:])    
        length = self.LENGTHS[opcode]
        args = [self.memory[self.memory_pointer + i] for i in range(1, length)]
        modes = op[:-2].zfill(length - 1)[::-1]
        locs = []
        vals = []
        for arg, mode in zip(args, modes):
            if mode == self.POSITION_MODE: 
                locs.append(arg)
                vals.append(self.memory[arg])
            elif mode == self.IMMEDIATE_MODE:
                locs.append(None)
                vals.append(arg)
            elif mode == self.RELATIVE_MODE:
                locs.append(arg + self.relative_base)
                vals.append(self.memory[arg + self.relative_base])

        return self.INSTRUCTIONS[opcode](opcode, length, locs, vals, self.memory)
        
    def run(self, inputs = []):
        while True:
            instruction = self.create_instruction()
            
            if instruction.opcode == self.HALT:
                break
            elif instruction.opcode == self.INPUT:
                value = instruction.call(inputs.pop(0))
            else:
                value = instruction.call()
            
            self.memory_pointer += instruction.length
            if instruction.opcode in [self.JUMP_IF_TRUE, self.JUMP_IF_FALSE] and value is not None:
                self.memory_pointer = value
            elif instruction.opcode == self.RELATIVE_BASE_OFFSET:
                self.relative_base += value
            elif instruction.opcode == self.OUTPUT:
                yield value

    @staticmethod
    def to_program(data):
        return defaultdict(int, enumerate(data))

Let's run some tests. This program's output should be a copy of the program itself! WTF 

In [14]:
data = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
program = IntcodeComputer.to_program(data)
output = IntcodeComputer(program).run()
list(output)

[109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99]

This program should make a giant integer. This is to make sure that our program can handle big ints.

In [15]:
data = [1102,34915192,34915192,7,4,7,99,0]
program = IntcodeComputer.to_program(data)
output = IntcodeComputer(program).run()
list(output)

[1219070632396864]

One more test, for old times sake.

In [16]:
data = [104,1125899906842624,99]
program = IntcodeComputer.to_program(data)
output = IntcodeComputer(program).run()
list(output)

[1125899906842624]

## Part 1:
Run the program with input `1`. This takes a few milliseconds to run. Not bad!

In [17]:
with open('day_09.txt') as f:
    data = [int(token) for token in f.readlines()[0].strip().split(',')]

In [18]:
%%time
program = IntcodeComputer.to_program(data)
output = IntcodeComputer(program).run([1])
list(output)

CPU times: user 1.34 ms, sys: 49 µs, total: 1.39 ms
Wall time: 1.39 ms


[2457252183]

## Part 2:
Run the same program, but with input `2`. This one takes longer, but still just a couple seconds. I guess we coded this puppy efficiently enough!

In [19]:
%%time
program = IntcodeComputer.to_program(data)
output = IntcodeComputer(program).run([2])
list(output)

CPU times: user 2.08 s, sys: 7.76 ms, total: 2.09 s
Wall time: 2.09 s


[70634]